In [1]:
import pandas as pd

In [3]:
df = pd.read_csv("Task 3 and 4_Loan_Data.csv")

In [4]:
df.head()

,customer_id,credit_lines_outstanding,loan_amt_outstanding,total_debt_outstanding,income,years_employed,fico_score,default
0,8153374,0,5221.545193,3915.471226,78039.38546,5,605,0
1,7442532,5,1958.928726,8228.752520,26648.43525,2,572,1
2,2256073,0,3363.009259,2027.830850,65866.71246,4,602,0
3,4885975,0,4766.648001,2501.730397,74356.88347,5,612,0
4,4700614,1,1345.827718,1768.826187,23448.32631,6,631,0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   customer_id               10000 non-null  int64  
 1   credit_lines_outstanding  10000 non-null  int64  
 2   loan_amt_outstanding      10000 non-null  float64
 3   total_debt_outstanding    10000 non-null  float64
 4   income                    10000 non-null  float64
 5   years_employed            10000 non-null  int64  
 6   fico_score                10000 non-null  int64  
 7   default                   10000 non-null  int64  
dtypes: float64(3), int64(5)
memory usage: 625.1 KB


In [6]:
df.describe()

,customer_id,credit_lines_outstanding,loan_amt_outstanding,total_debt_outstanding,income,years_employed,fico_score,default
count,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,4.974577e+06,1.461200,4159.677034,8718.916797,70039.901401,4.552800,637.557700,0.185100
std,2.293890e+06,1.743846,1421.399078,6627.164762,20072.214143,1.566862,60.657906,0.388398
min,1.000324e+06,0.000000,46.783973,31.652732,1000.000000,0.000000,408.000000,0.000000
25%,2.977661e+06,0.000000,3154.235371,4199.836020,56539.867903,3.000000,597.000000,0.000000
50%,4.989502e+06,1.000000,4052.377228,6732.407217,70085.826330,5.000000,638.000000,0.000000
75%,6.967210e+06,2.000000,5052.898103,11272.263740,83429.166133,6.000000,679.000000,0.000000
max,8.999789e+06,5.000000,10750.677810,43688.784100,148412.180500,10.000000,850.000000,1.000000


In [12]:
#prepare data for modelling, so separate inputs and targets
X = df.drop("default", axis=1)  # all inputs
y = df["default"]               # target

In [13]:
#train-test-split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [23]:
#train logistic regression
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'lbfgs'
,max_iter,1000
,multi_class,'deprecated'


In [24]:
#predict probablity of default pd
pd_predictions = model.predict_proba(X_test)[:, 1]
pd_predictions[:5]

array([5.46575528e-12, 1.46052773e-03, 1.00000000e+00, 4.74605484e-14,
       1.39664749e-12])

In [25]:
pd_predictions

array([5.46575528e-12, 1.46052773e-03, 1.00000000e+00, ...,
       4.90848260e-13, 8.98123070e-12, 1.06328232e-11], shape=(3000,))

In [26]:
#calculate expected loss 
def expected_loss(loan_amount, pd, recovery_rate=0.10):
    lgd = 1 - recovery_rate
    return loan_amount * pd * lgd

In [27]:
#test expected loss fucntion
sample_pd = 0.25
loan_amount = 500000
expected_loss(loan_amount, sample_pd)


112500.0

In [28]:
#predict expected loss
def predict_expected_loss(borrower_data, model, recovery_rate=0.10):
    """
    borrower_data: pandas DataFrame with same columns as training X
    """
    pd = model.predict_proba(borrower_data)[:, 1][0]
    loan_amount = borrower_data["loan_amount"].values[0]
    return expected_loss(loan_amount, pd, recovery_rate)

In [29]:
#basic model evaluation - logistic regression
from sklearn.metrics import roc_auc_score
auc = roc_auc_score(y_test, pd_predictions)
auc

0.9999812075148893

In [30]:
#model evaluation -  decision tree
from sklearn.tree import DecisionTreeClassifier

tree = DecisionTreeClassifier(max_depth=4)
tree.fit(X_train, y_train)

tree_pd = tree.predict_proba(X_test)[:, 1]
roc_auc_score(y_test, tree_pd)

0.998419552002189

In [32]:
#look for leakages as model is suspiciously good
df.corr()["default"].sort_values(ascending=False)

default                     1.000000
credit_lines_outstanding    0.862815
total_debt_outstanding      0.758868
loan_amt_outstanding        0.098978
income                      0.016309
customer_id                 0.006927
years_employed             -0.284506
fico_score                 -0.324515
Name: default, dtype: float64

In [33]:
#drop suspicous columns
leakage_cols = [
    "past_default",
    "loan_status",
    "days_past_due",
    "written_off"
]

X = df.drop(["default"] + leakage_cols, axis=1, errors="ignore")


In [35]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [36]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'lbfgs'
,max_iter,1000
,multi_class,'deprecated'


In [37]:
#basic model evaluation - logistic regression - after dropping suspicious columns
from sklearn.metrics import roc_auc_score
auc = roc_auc_score(y_test, pd_predictions)
auc

0.9999812075148893

In [38]:
#model evaluation -  decision tree
from sklearn.tree import DecisionTreeClassifier

tree = DecisionTreeClassifier(max_depth=4)
tree.fit(X_train, y_train)

tree_pd = tree.predict_proba(X_test)[:, 1]
roc_auc_score(y_test, tree_pd)

0.998419552002189

In [39]:
pd.Series(model.coef_[0], index=X.columns).sort_values()

years_employed             -2.662816e+00
fico_score                 -2.224798e-02
income                     -2.213120e-04
customer_id                 5.477931e-09
loan_amt_outstanding        4.059694e-04
total_debt_outstanding      1.011992e-03
credit_lines_outstanding    7.213189e+00
dtype: float64